## CCTVストリーム画像から災害検知するエージェント
### 🧩 システム構成（MVP）
- graph TD
- A[CCTV画像ストリーム] --> B[画像キャプチャ & 前処理]
- B --> C[UI-TARS-1.5: Image-to-Text]
- C --> D[災害キーワード、災害スコア判定モジュール]
- D --> E[位置特定（カメラID → 地図）]
- E --> F[被害要約生成]
- F --> G[自然言語レポート生成]

#### 1. 📸 画像キャプチャ & 前処理
- 各CCTVから定期的に画像を取得（例：1分ごと）
- 解像度調整、ノイズ除去、タイムスタンプ付与

##### 🔧 必要なツール
- | ツール | 用途 | 
- | ffmpeg | ストリームからフレームを抽出 | 
- | OpenCV | Pythonで画像処理と保存 | 
##### インストール
- pip install yt-dlp opencv-python
- ⇒Successfully installed opencv-python-4.12.0.88 yt-dlp-2025.6.30 
- ▼Not well：sudo apt install ffmpeg
##### sudo無効の仮想環境での対処法
- そのメッセージは、お使いの環境（一部の仮想環境）で sudo 権限が無効化されていることを示しています。以下に、状況別の対処法を整理しました：
- 🛠️ 対処法：sudo が無効な環境で ffmpeg を使うには？
- ✅ 方法①：ffmpeg をバイナリで直接インストール（推奨）
- 公式ffmpegビルドサイト または FFmpeg GitHub から静的バイナリをダウンロード、例（Linux x86_64）:
- 実行ファイルをパスに追加（例：~/.local/bin）
- 動作確認
##### ffmpeg.exe Windows用
- ffmpeg（拡張子なし）は Linux用の実行バイナリ
- Windowsでは .exe 拡張子のある Windowsネイティブビルド が必要
✅ 解決策：Windows向けの正しい ffmpeg.exe を入手する
以下の手順で、Windows用の ffmpeg.exe を確実に取得できます：
🔗 ステップ1：公式Windowsビルドサイトにアクセス
👉 https://www.gyan.dev/ffmpeg/builds/
📦 ステップ2：次のいずれかをダウンロード
- ffmpeg-release-essentials.zip（軽量で必要最小限）、解凍し、exeを確認
✅フォルダーを作成
- New-Item -ItemType Directory -Path "C:\Tools\ffmpeg\bin" -Force
✅環境変数 PATH に追加（永続化）
- $env:Path += ";C:\Tools\ffmpeg\bin"
✅ 動作確認
- ffmpeg -version → バージョン情報が表示されれば成功です！
- ffmpeg version 7.1.1-essentials_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers

### 🎥 ストリーム画像取得スクリプト（Windows環境向け）
- CCTVストリームからの画像取得スクリプトの整備に進みましょう。
- 関東地方整備局より、ライブ映像が試験配信されている。
- 10秒ごとに、カメラが切り替わっている。
- 次の「河川YouTubeライブ映像」を対象に、10秒ごとに静止画像を保存できるMVPコードを用意します。
- 【試験配信】関東地方整備局　那珂川・久慈川 ライブ映像

✅ 必要なツール
- | ツール | 用途 | 
- | yt-dlp | YouTubeライブのストリームURLを取得 | 
- | ffmpeg.exe | ストリームから静止画を抽出 | 
- | PowerShell | スケジュール＆保存処理 | 

- PowerShellスクリプトをPythonから実行できるように修正しました。
新しいCell In[1]は、subprocessを使ってPowerShellスクリプトを呼び出すPythonコードになっています。
- このまま実行すれば、PowerShellの処理がPythonから開始されます。

In [ ]:
import subprocess
from datetime import datetime, timedelta
import time

List_of_area_name = ["NakaKuji", "TamaTsuruSagami", "ToneKarasu", "Watarase"]
List_of_youtube_url = [
    "https://www.youtube.com/live/bgHmONs2gKM?si=xJ0_h4P756H_9Y27",
    "https://www.youtube.com/live/QUozBiadHV8?si=7QAFRkBkKha3VDcy",
    "https://www.youtube.com/live/IyE0H7nKP68?si=vaTbIU5cE8VFtQeg",
    "https://www.youtube.com/live/R3KlRdrNyGs?si=0xb1_VLFnrvjK7iF"
]

# スケジュール開始・終了時刻
start_time = datetime.combine(datetime.today(), datetime.strptime("17:15", "%H:%M").time())
end_time = datetime.combine(datetime.today(), datetime.strptime("22:00", "%H:%M").time())

# 初回実行時刻を開始時刻に設定
next_run = start_time

# 現在時刻が終了時刻を過ぎるまでループ
while next_run <= end_time:
    # 実行時刻まで待機（秒単位で短くループ）
    while datetime.now() < next_run:
        time.sleep(1)
    # 画像キャプチャ処理
    for area, youtube_url in zip(List_of_area_name, List_of_youtube_url):
        save_path = f"C:\\Users\\yasun\\PyTorch\\CCTVDisasterAgent\\{area}Frames"
        powershell_script = fr'''
        $youtubeURL = "{youtube_url}"
        $savePath   = "{save_path}"
        $ffmpegPath = "C:\\Tools\\ffmpeg\\bin\\ffmpeg.exe"
        $interval   = 5
        $Num_of_Captures = 5
        New-Item -ItemType Directory -Path $savePath -Force | Out-Null
        for ($i = 0; $i -lt $Num_of_Captures; $i++) {{
            $timestamp = Get-Date -Format "yyyyMMdd_HHmmss"
            $imageFile = "$savePath\\frame_$timestamp.jpg"
            $streamURL = yt-dlp -g $youtubeURL
            & "$ffmpegPath" -y -i "$streamURL" -frames:v 1 "$imageFile"
            Write-Host "✅ Saved: $imageFile"
            Start-Sleep -Seconds $interval
        }}
        '''
        subprocess.run(['powershell', '-Command', powershell_script])
    # 次回実行時刻を5分後に更新
    next_run += timedelta(minutes=5)

#### (FUTURE)災害スコアを時系列に可視化
- 地点別に災害のハザードの高まりを可視化し、備える行動の判断材料とする

#### (PEND) 📍 発生位置の特定
- 災害あり判定のカメラIDを特定
- カメラ位置のGPS座標のマッピングテーブルを用意
- (PEND)発生地点を地図上にプロット（例：Plotly, Leaflet）